In [2]:
import config
import mysql.connector 
import requests
import json
from bs4 import BeautifulSoup
from mysql.connector import errorcode
import time

cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'Movies'
)
cursor = cnx.cursor()

In [2]:
# db_name = 'Movies'

# def create_database(cursor, database):
#     try:
#         cursor.execute(
#             "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
#     except mysql.connector.Error as err:
#         print("Failed creating database: {}".format(err))
#         exit(1)
        
#     #above is python code that is going to connect to SQL queries w/ cursor.execute
#     #cursor knows to take SQL query and send it to server to run code
#     #try & except is error handling within python
#     #format takes in database and tries to run it
#     #if failed, except will tell you error and why it failed
#     #above is a function to create a database

# try:
#     cursor.execute("USE {}".format(db_name))
# except mysql.connector.Error as err:
#     print("Database {} does not exists.".format(db_name))
#     if err.errno == errorcode.ER_BAD_DB_ERROR:
#         create_database(cursor, db_name)
#         print("Database {} created successfully.".format(db_name))
#         cnx.database = db_name
#     else:
#         print(err)
#         exit(1)

In [3]:
cursor = cnx.cursor()
# TABLES = {} #use dictionary to save diff create statements for each table
# TABLES['The_Movie_Db'] = (
#     "  CREATE TABLE The_Movie_Db ("
#     "  id int(11) NOT NULL AUTO_INCREMENT primary key,"
#     "  movie_name varchar(200) not null,"
#     "  budget int,"
#     "  revenue int,"
#     "  rating float,"
#     "  release_date date"    
# ") ENGINE=InnoDB")


# TABLES = {} #use dictionary to save diff create statements for each table
# TABLES['The_Movie_Db_Genres'] = (
#     "  CREATE TABLE The_Movie_Db_Genres ("
#     "  id int(11) NOT NULL AUTO_INCREMENT primary key,"
#     "  movie_name varchar(200) not null,"
#     "  genres varchar(200)"
# ") ENGINE=InnoDB")


# for table_name in TABLES:
#     table_description = TABLES[table_name]
#     try:
#         print("Creating table {}: ".format(table_name), end='')
#         cursor.execute(table_description)
#     except mysql.connector.Error as err:
#         if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
#             print("already exists.")
#         else:
#             print(err.msg)
#     else:
#         print("OK")


Creating table The_Movie_Db_Genres: already exists.


# get ids from the movie db >>>>scripted 

In [37]:
list_of_id = []

for i in range(1,11):
    url = "https://www.themoviedb.org/movie?page={}".format(i)
#     print(url)
    page = requests.get(url)

    soup = BeautifulSoup(page.content, 'html.parser')

    movies = soup.find(class_="results flex results_poster_card")

    movie_detail = movies.find_all(class_="flex")

    for i in range(len(movie_detail)):
        id_string = movie_detail[i].find('a')
        id_int = int(id_string['href'].split('/')[2])
        list_of_id.append(id_int)



In [39]:
len(list_of_id)

200

# call the movie db to get movies info >>>> with api key

In [41]:
api_key = '2f848bb1906d7277820e69e3fa9158a5'
# movies_lists = []
for i in list_of_id:
    url = 'https://api.themoviedb.org/3/movie/{}?api_key=2f848bb1906d7277820e69e3fa9158a5'.format(i)
#     print(url)
    response = requests.get(url)

    data = response.json()
    keys = ['title', 'budget', 'revenue' ,'vote_average','release_date']
    parsed_results = [{key: (data[key] if key in data else 'Null') for key in keys}]

    
    for movie in parsed_results:
    
        movie_tuple = (movie['title'],movie['budget'], movie['revenue']
                       , movie['vote_average'], movie['release_date'])
#         movies_lists.append(movie_tuple)
       
        insert_stmt = "INSERT INTO The_Movie_Db ( movie_name, budget, revenue, rating, release_date) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(insert_stmt, movie_tuple)
        cnx.commit()
        time.sleep(1)
